In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Input: numpy array of images and number of gray levels to quantize the images down to
# Output: numpy array of images, each with only n_colors gray levels
def quantization(imgs_arr, n_colors=4):
	img_size = imgs_arr[0].shape
	res = []
	
	for img in imgs_arr:
		X = img.reshape(img_size[0] * img_size[1], 1)
		km = KMeans(n_clusters=n_colors)
		km.fit(X)
		
		img_compressed = km.cluster_centers_[km.labels_]
		img_compressed = np.clip(img_compressed.astype('uint8'), 0, 255)

		res.append(img_compressed.reshape(img_size[0], img_size[1]))
	
	return np.array(res)

# Input: A path to a folder and formats of images to read
# Output: numpy array of grayscale versions of images read from input folder, and also a list of their names
def read_dir(folder, formats=(".jpg", ".png")):
	image_arrays = []
	lst = [file for file in os.listdir(folder) if file.endswith(formats)]
	for filename in lst:
		file_path = os.path.join(folder, filename)
		image = cv2.imread(file_path)
		gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		image_arrays.append(gray_image)
	return np.array(image_arrays), lst

# Input: image object (as numpy array) and the index of the wanted bin (between 0 to 9)
# Output: the height of the idx bin in pixels
def get_bar_height(image, idx):
	# Assuming the image is of the same pixel proportions as images supplied in this exercise, the following values will work
	x_pos = 70 + 40 * idx
	y_pos = 274
	print(image[y_pos, x_pos])
	while image[y_pos, x_pos] == 245:
		y_pos -= 1
	return 274 - y_pos

# Sections c, d
# Remember to uncomment compare_hist before using it!

import matplotlib.pyplot as plt

#  Input: 2 accumulated histograms
# Output: their EMD
def calculate_EMD(C_a, C_b):
	return np.sum(np.abs(C_a - C_b))

def compare_hist(src_image, target):
	target_hist = cv2.calcHist([target], [0], None, [256], [0, 256]).flatten()
	target_cumsum = np.cumsum(target_hist, dtype = np.float32)

	height, width = target.shape
	windows = np.lib.stride_tricks.sliding_window_view(src_image, (height, width))

	# offset range of the windows
	# x and y are the actual offset
	hh, ww = windows.shape[:2]
	for x in range(25, 50):
		for y in range(100, 145):
			window = windows[y, x]
			window_hist = cv2.calcHist([window], [0], None, [256], [0, 256]).flatten()
			window_cumsum = np.cumsum(window_hist, dtype = np.float32)

			emd = calculate_EMD(target_cumsum, window_cumsum)
			if emd < 260:
				return True
	return False

In [ ]:
def top_num_on_scale(src_image, numbers):
    i = len(numbers) - 1
    while i >= 0:
        res = compare_hist(src_image, numbers[i])
        if res is True:
            return i
        i -= 1
    return -1

In [ ]:
images, names = read_dir('data')
numbers, numberNames = read_dir('numbers')

target_img = numbers[0]
img = images[1]

# count = 1
# for img in images:
# 	print(f'Picture: {count} - Scale: {top_num_on_scale(img, numbers)}')
# 	count += 1


quantisized_imgs = quantization(images, 2)

img = quantisized_imgs[0]

# img_rgb = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
# plt.imshow(img_rgb)
# plt.axis('off')  # Hide axes
# plt.title('Image Display')
# plt.show()

In [ ]:
bars = np.zeros([10])

for i in range(len(bars)):
    bars[i] = get_bar_height(img, i)
    print(bars[i])